In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier

class StockManager:
    """
    Manage your stock, get pick list with 4 picking method: FIFO, LIFO, FEFO, by EXP date
    
    Parameters
    ----------
    path: path to your stock data file in csv format
    
    Attributes
    ----------
    stock: initial dataframe of stock data, it will not change at all
    
    remain: dataframe of current stock, it will change after each picking
    
    pick_list: dataframe of current pick list for inputted sales order
    
    pick_history: dataframe of pick history
    
    """
    
    def __init__(self, path: str):        
        self.stock = pd.read_csv(path)
        self.stock.good_receipt_time = pd.to_datetime(self.stock.good_receipt_time, format='%Y/%m/%d')
        self.stock.expiry_date_time = pd.to_datetime(self.stock.expiry_date_time, format='%Y/%m/%d')
        self.__createDF()
        
    def __createDF(self):
        self.remain = self.stock.copy()
        self.remain['floor'] = [i[-2:-1] for i in self.remain.location]
        self.remain['floor'] = self.remain['floor'].astype(int)
        self.__formatRemainingStock()
        
        self.pick_list = pd.DataFrame(columns=self.stock.columns)
        self.pick_list.rename(columns={'stock_qty': 'picking_quantity'}, inplace=True)
        self.pick_list[['order_no', 'created_at']] = ""
        self.pick_history = self.pick_list.copy()
            
    def __extractDate(self, text: str):
        """extract required EXP date for picking method "by EXP date"        
        """
        match = re.search(r'\d{4}-\d{2}-\d{2}', text)
        if match == None:
            raise ValueError("Can not extract date, check date of pick rule on pick list")
        date = datetime.strptime(match.group(), '%Y-%m-%d').date()
        return date
    
    def __getSalesOrder(self, path: str):
        """get Sales Order information from data file in .csv format
        """
        so_columns = ['order_no', 'item_code', 'order_qty', 'pick_rule']
        self.sales_order = pd.read_csv(path, names=so_columns, skiprows=1)
        pick_rules = ['FIFO', 'FEFO', 'LIFO', 'EXP']        
        self.sales_order['exp_date'] = [np.datetime64('NaT') if rule in pick_rules \
                                        else pd.to_datetime(self.__extractDate(rule)) \
                                        for rule in self.sales_order.pick_rule]
        self.sales_order.loc[~self.sales_order.exp_date.isna(), 'pick_rule'] = 'EXP'

    def __clearSalesOrder(self):
        self.sales_order.drop(index=self.sales_order.index, inplace=True)
        
    def __checkSO_Qty(self, order: pd.DataFrame):
        """check if current stock quantity can fulfill Sales Order
        Parameters
        ----------
        order: Sales Order in dataframe
        
        Return
        ------
        bool True/False
        """
        
        check=True
        
        # check code
        code_valid = order.item_code.isin(self.remain.item_code)
        if not all(code_valid):
            print('Invalid item code found')
            print(order.item_code.loc[code_valid])
            check=False
        
        remain_agg = self.remain.groupby(by='item_code').sum()
        order_agg = order.groupby(by='item_code').sum()

        #check quantity all pick type
        for order_code in order_agg.index:
            order_qty = order_agg.order_qty[order_code]
            remain_qty = remain_agg.loc[order_code, 'stock_qty']
            if order_qty > remain_qty:
                print('Code {} not enough qty'.format(order_code))
                check=False
        
        #check quantity EXP pick type
        remain_agg_EXP = self.remain.groupby(by=['item_code', 'expiry_date_time']).sum()
        order_agg_EXP = order.groupby(by=['item_code', 'exp_date']).sum()
        for order_code, exp_date in order_agg_EXP.index:
            if not pd.isnull(exp_date):
                order_qty_EXP = order_agg_EXP.loc[(order_code, exp_date), 'order_qty']
                remain_qty_EXP = remain_agg_EXP.loc[(order_code, exp_date), 'stock_qty']
                if  order_qty_EXP > remain_qty_EXP:
                    print('Code {} not have enough qty for EXP date {}'\
                        .format(order_code, exp_date))
                    check=False
            
        return check
    
    def __getSortedDataForPickRule(self, order: pd.Series, rule: str):
        """
        Sort stock data to match pick rule requirement. What needed to be picked first
        will be on top
        
        Parameters:
        ----------
        order (pd.Series): one request in Sales Order (one row in Sales Order)
        rule (str): pick rule for that Order

        Raises:
        ------
        Exception: picking rule/method did not match what's defined

        Returns:
        -------
        dataframe of sorted stock for pick rule
        """
        avail_item = self.remain.loc[self.remain.item_code == order.item_code]        
        if rule=='FIFO':
            avail_item = avail_item.sort_values(by=['good_receipt_time', 'floor', 'expiry_date_time'])
        elif rule =='FEFO':
            avail_item = avail_item.sort_values(by=['expiry_date_time', 'floor', 'good_receipt_time'])
        elif rule =='LIFO':
            avail_item = avail_item.sort_values(by=['good_receipt_time', 'floor', 'expiry_date_time'], ascending=[False, True, True])
        elif rule =='EXP':
            avail_item = avail_item.loc[avail_item.expiry_date_time == order.exp_date].sort_values(by=['good_receipt_time', 'floor'])
        else: raise Exception("Sorry, pick rule not valid")
        avail_item['cum_sum'] = avail_item.stock_qty.cumsum()
        return avail_item
        
    def __pickByRow(self, order: pd.Series):
        """put picking data into pick_list
        Parameters:
        ----------
        order (pd.Series): one request in Sales Order (one row in Sales Order)
        """

        sorted_stock = self.__getSortedDataForPickRule(order, rule=order.pick_rule)
        order_qty = order.order_qty
        
        sorted_stock.rename(columns={'stock_qty':'picking_quantity'}, inplace=True)
        df_pick = pd.DataFrame(columns=self.pick_list.columns)
                
        for i, idx in enumerate(sorted_stock.index):
            if sorted_stock.cum_sum[idx] < order_qty:
                df_pick = df_pick.append(sorted_stock.iloc[i,0:5], ignore_index = True)
                self.remain.loc[idx, 'stock_qty'] = 0
            else:
                remain_to_pick = order_qty - df_pick.picking_quantity.sum()
                df_pick = df_pick.append(sorted_stock.iloc[i,0:5], ignore_index = True)
                df_pick.loc[i, 'picking_quantity'] = remain_to_pick
                self.remain.loc[idx, 'stock_qty'] = sorted_stock.cum_sum[idx] - order_qty
                break
            
        df_pick.order_no = order.order_no
        df_pick.created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        self.pick_list = self.pick_list.append(df_pick, ignore_index=True)

        return
        
    def __formatRemainingStock(self):
        """format remaining_report to match output require"""
        remaining_cols = ['item_code', 'location', 'good_receipt_time', 'expiry_date_time', 'stock_qty']
        self.remain_report = self.remain[remaining_cols].loc[self.remain.stock_qty > 0].reset_index(drop=True).copy()
        self.remain_report.rename(columns={'stock_qty':'remain_quantity'}, inplace=True)

    def __savePickHistory(self):
        self.pick_history = pd.concat([self.pick_history, self.pick_list])
        self.pick_history.reset_index(drop=True, inplace=True)
    
    def __clearOldPickList(self):
        self.pick_list.drop(self.pick_list.index, inplace=True)
        
    def __checkQtyAfterPicking(self):
        """check if total stock qty are still the same"""
        print("Check quantity after picking")
        print(self.stock.stock_qty.sum() - self.remain.stock_qty.sum() \
                == self.pick_history.picking_quantity.sum())
        
    def pickBySalesOrder(self, path: str):
        """compute pick list
        
        Parameters:
        ----------
        path: path to Sales Order file in csv format
        """

        self.__getSalesOrder(path)
        self.__clearOldPickList()
        
        if self.__checkSO_Qty(self.sales_order):
            for _, order in self.sales_order.iterrows():
                self.__pickByRow(order)
                
            self.__savePickHistory()
            self.__checkQtyAfterPicking()
            self.__formatRemainingStock()
            display(self.pick_list)
        else:
            print("Picking failed!")
                
        self.__clearSalesOrder()

In [5]:
stock_path = './input/Data.csv'
sales_order_path = './input/sales_order.csv'

stockMgt = StockManager(stock_path)
stockMgt.pickBySalesOrder(sales_order_path)

Check quantity after picking
True


,item_code,picking_quantity,location,good_receipt_time,expiry_date_time,order_no,created_at
0,SA,11,0321B,2020-05-30 16:03:00,2020-06-15,1,2022-08-26 01:07:17
1,SA,3,0304A,2020-07-16 09:55:00,2020-08-05,1,2022-08-26 01:07:17
2,SA,9,1419B,2020-07-16 09:55:00,2020-08-05,1,2022-08-26 01:07:17
3,SA,4,0302A,2020-07-22 13:52:00,2020-08-21,1,2022-08-26 01:07:17
4,BH,59,1603D,2020-06-15 08:28:00,2020-06-26,2,2022-08-26 01:07:17
5,BH,28,1603D,2020-06-15 08:28:00,2020-07-15,2,2022-08-26 01:07:17
6,BH,38,1823B,2020-07-01 16:31:00,2020-07-28,2,2022-08-26 01:07:17
7,BH,3,0513B,2020-08-15 08:28:00,2020-08-30,2,2022-08-26 01:07:17
8,BH,29,1611B,2020-08-15 08:28:00,2020-09-06,2,2022-08-26 01:07:17
9,E2,9,0218A,2020-10-28 19:34:00,2020-11-14,3,2022-08-26 01:07:17


In [3]:
stockMgt.remain_report

,item_code,location,good_receipt_time,expiry_date_time,remain_quantity
0,SA,0304A,2020-07-24 16:42:00,2020-08-20,2
1,SA,0307A,2020-08-01 09:50:00,2020-08-13,16
2,SA,0307A,2020-08-01 09:50:00,2020-08-20,4
3,BH,1611B,2020-08-15 08:28:00,2020-09-11,53
4,BH,1611B,2020-08-15 08:28:00,2020-09-06,33
5,BH,0218A,2020-09-01 14:16:00,2020-09-18,10
6,BH,0220A,2020-09-01 14:16:00,2020-09-28,9
7,E2,0628B,2020-10-15 13:26:00,2020-11-09,3
8,YAA,0212A,2020-08-27 16:06:00,2020-09-13,2
9,YAA,0214A,2020-08-27 16:06:00,2020-09-21,3


In [4]:
# nếu tục pick lần 2 với cùng sales order đó
stockMgt.pickBySalesOrder(sales_order_path)

Code BH not enough qty
Code E2 not enough qty
Code SA not enough qty
Code YAA not have enough qty for EXP date 2020-11-01 00:00:00
Picking failed!
